In [53]:
import numpy as np
import pandas as pd
import datetime


x = str(datetime.datetime(2020,3,5,1,0,0)).split()
y = str(datetime.datetime(2020,3,5,4,0)).split()
z = str(datetime.datetime(2020,3,5,7,0,0)).split()
ab = str(datetime.datetime(2020,3,5,0,0,0)).split()
cd = str(datetime.datetime(2020,3,5,3,0,0)).split()
print(x)

df = pd.DataFrame({'date time':[x,z,ab,y,cd]})
display(df)

df = df.sort_values(by='date time',ascending=True)
display(df)




['2020-03-05', '01:00:00']


,date time
0,"[2020-03-05, 01:00:00]"
1,"[2020-03-05, 07:00:00]"
2,"[2020-03-05, 00:00:00]"
3,"[2020-03-05, 04:00:00]"
4,"[2020-03-05, 03:00:00]"


,date time
2,"[2020-03-05, 00:00:00]"
0,"[2020-03-05, 01:00:00]"
4,"[2020-03-05, 03:00:00]"
3,"[2020-03-05, 04:00:00]"
1,"[2020-03-05, 07:00:00]"


In [69]:
import datetime
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt 
from suds.client import Client


sample_time = str(datetime.datetime(2020,3,5,1,0,0)).split()
#display(sample_time)


def get_input():
    user_in_fir = input("Enter from/to time periods and mySite: ").split()
    #from time period, to time period, mySite 
    return user_in_fir[0],user_in_fir[1],user_in_fir[2]


def get_timeseries_arr(Date_begin, Date_end):
    year_b,month_b,day_b,hour_b,min_b = Date_begin.split("/")[2],Date_begin.split("/")[1],Date_begin.split("/")[0],0,0
    year_e,month_e,day_e,hour_e,min_e = Date_end.split("/")[2],Date_end.split("/")[1],Date_end.split("/")[0],23,45 #data recorded every 15 mins
    #print(year_b,month_b,day_b,hour_b,min_b)
    #print(year_e,month_e,day_e,hour_e,min_e)
    start_dt = datetime.datetime(int(year_b),int(month_b),int(day_b),int(hour_b),int(min_b),0)
    end_dt = datetime.datetime(int(year_e),int(month_e),int(day_e),int(hour_e),int(min_e),0)
    index = 0
    x = start_dt
    all_time_series = []
    while not x == end_dt:
        x = x + timedelta(minutes = 15)
        all_time_series.append(x.strftime('%Y-%m-%d %H:%M:00'))
    #print(all_time_series)
    return all_time_series

def makeday(num):
    if int(num) <10:
        return'0'+num

def newformat(Date_begin, Date_end):
    if "/" in Date_begin:
        Date_begin_new = (str(Date_begin).split("/")[2]+"-"+makeday(str(Date_begin).split("/")[1])+"-"+makeday(str(Date_begin).split("/")[0]))
        Date_end_new = (str(Date_end).split("/")[2]+"-"+makeday(str(Date_end).split("/")[1])+"-"+makeday(str(Date_end).split("/")[0]))
    #print(Date_begin_new,Date_end_new)
    return Date_begin_new,Date_end_new

def fill_NAN(time_Series_arr, Dict):
    Keys_arr = list(Dict.keys())
    for i in range(len(time_Series_arr)):
        if not(time_Series_arr[i] in Keys_arr):
            Dict[time_Series_arr[i]] = float('nan')
            #print("Adding")
        #else:
            #print("FOUND")
            
            
            
            


Date_begin, Date_end, site = get_input()
time_Series_arr = get_timeseries_arr(Date_begin, Date_end)

wsdlURL = 'http://hydroportal.cuahsi.org/nwisuv/cuahsi_1_1.asmx?WSDL'
siteCode = 'NWISUV:'+site

variableCode_wat_c = 'NWISUV:00010'
variableCode_air_f = 'NWISUV:00021'
variableCode_winspd_mh = 'NWISUV:00035'
variableCode_disch_c3s = 'NWISUV:00060'

Dict_wat_c = {}
Dict_air_f = {}
Dict_winspd_mh = {} 
Dict_disch_c3s = {}


Date_begin, Date_end = newformat(Date_begin, Date_end)


# Create a new object named "NWIS" for calling the web service methods
NWIS = Client(wsdlURL).service

#Every one-quarter hour data for these dates!
response_wat_c = NWIS.GetValuesObject(siteCode, variableCode_wat_c, Date_begin, Date_end) 
response_air_f = NWIS.GetValuesObject(siteCode, variableCode_air_f, Date_begin, Date_end) 
response_winspd_mh = NWIS.GetValuesObject(siteCode, variableCode_winspd_mh, Date_begin, Date_end) 
response_disch_c3s = NWIS.GetValuesObject(siteCode, variableCode_disch_c3s, Date_begin, Date_end)



if not len(response_wat_c.timeSeries[0]) == 0:
    values_wat_c = response_wat_c.timeSeries[0].values[0].value
    for v in values_wat_c:
            Dict_wat_c[v._dateTime.strftime('%Y-%m-%d %H:%M:%S')] = float(v.value)
if not len(response_air_f.timeSeries[0]) == 0:
    values_air_f = response_air_f.timeSeries[0].values[0].value
if not len(response_winspd_mh.timeSeries[0]) == 0:
    values_winspd_mh = response_winspd_mh.timeSeries[0].values[0].value
if not len(response_disch_c3s.timeSeries[0]) == 0:
    values_disch_c3s = response_disch_c3s.timeSeries[0].values[0].value


#display(Dict_wat_c)
display(len(Dict_wat_c))
fill_NAN(time_Series_arr,Dict_wat_c)
display(len(time_Series_arr))
display(len(Dict_wat_c))
display(Dict_wat_c)



#df = pd.DataFrame({'water in celcius':list(Dict_wat_c.values())}, index = time_Series_arr)
#display(df)




dates = [x.split()[0] for x in time_Series_arr]
times = [x.split()[1] for x in time_Series_arr]

#df = pd.DataFrame({'date':dates, 'time':times})
#display(df)







#1/4/2020 7/4/2020 08048000





Enter from/to time periods and mySite: 1/4/2020 7/4/2020 08048000


657

671

672

{'2020-04-01 00:00:00': 17.8,
 '2020-04-01 00:15:00': 17.8,
 '2020-04-01 00:30:00': 17.8,
 '2020-04-01 00:45:00': 17.7,
 '2020-04-01 01:00:00': 17.7,
 '2020-04-01 01:15:00': 17.7,
 '2020-04-01 01:30:00': 17.6,
 '2020-04-01 01:45:00': 17.6,
 '2020-04-01 02:00:00': 17.6,
 '2020-04-01 02:15:00': 17.5,
 '2020-04-01 02:30:00': 17.5,
 '2020-04-01 02:45:00': 17.5,
 '2020-04-01 03:00:00': 17.4,
 '2020-04-01 03:15:00': 17.4,
 '2020-04-01 03:30:00': 17.4,
 '2020-04-01 03:45:00': 17.3,
 '2020-04-01 04:00:00': 17.3,
 '2020-04-01 04:15:00': 17.2,
 '2020-04-01 04:30:00': 17.2,
 '2020-04-01 04:45:00': 17.2,
 '2020-04-01 05:00:00': 17.1,
 '2020-04-01 05:15:00': 17.1,
 '2020-04-01 05:30:00': 17.1,
 '2020-04-01 05:45:00': 17.0,
 '2020-04-01 06:00:00': 17.0,
 '2020-04-01 06:15:00': 17.0,
 '2020-04-01 06:30:00': 17.0,
 '2020-04-01 06:45:00': 17.0,
 '2020-04-01 07:00:00': 17.0,
 '2020-04-01 07:15:00': 17.0,
 '2020-04-01 07:30:00': 17.0,
 '2020-04-01 07:45:00': 17.0,
 '2020-04-01 08:00:00': 17.0,
 '2020-04-